In [9]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.clustering import KMeans
from pyspark.mllib.clustering import KMeans, KMeansModel

In [ ]:
cwd = os.getcwd()
print(cwd)
for part in cwd.split('/'):
    if part.startswith('edureka'):
        user_id = part.title()
user_id

app_name = '{0} : K-means clustering'.format(user_id)
app_name

In [ ]:
spark = SparkSession.builder.appName(app_name).getOrCreate()
sparkContext = spark.sparkContext
sqlContext = SQLContext(sparkContext)

In [ ]:
def get_hdfs_filepath(file_name):
    my_hdfs = '/common4all/{0}/may_batch_data/'.format(user_id.lower())
    return os.path.join(my_hdfs, file_name)

In [ ]:
# BOSTON_CSV = get_hdfs_filepath('boston_data.csv')
# BOSTON_CSV
datadir = '/common4all/edureka_200115/may_batch_data/'

In [8]:
# val schema = StructType(Array(
# StructField("dt", StringType, true),
# StructField("lat", DoubleType, true),
# StructField("lon", DoubleType, true),
# StructField("base", StringType, true)))

schema = StructType([
    StructField("dt", StringType()),
    StructField("lat", DoubleType()),
    StructField("lon", DoubleType()),
    StructField("base", StringType())            
])

df = spark.read.option("header", "true").schema(schema).csv(datadir + "uber-raw-data-apr14.csv")
df.show(5,False)
featureCols = ["lat", "lon"]

# VectorAssembler is a transformer that combines a given list of columns into a single vector column.
# It is useful for combining raw features and features generated by different feature transformers into a single
# feature vector, in order to train ML models like logistic regression and decision trees.
# assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

df2 = assembler.transform(df)
df2.show(5)
kmeans = KMeans(k=5, seed=1)
model = kmeans.fit(df2.select('features'))

transformed = model.transform(df2)
transformed.show() 
# kmeans = KMeans(k=2, seed=1)  # 2 clusters here
# model = kmeans.fit(new_df.select('features'))

# val Array(trainingData, testData) = df2.randomSplit(Array(0.7, 0.3), 8743)
## kmeans = KMeans(k=2, seed=1)
# kmeans = new KMeans().setK(10).setFeaturesCol("features").setPredictionCol("prediction")
# model = kmeans.fit(df2)
## model = kmeans.fit(df2.select('features'))

print("Final Centers: ")
## model.clusterCenters.foreach(println)
# clusters = KMeans.train(rdd, 2, maxIterations=10, initializationMode="random")

+----------------+-------+--------+------+
|dt              |lat    |lon     |base  |
+----------------+-------+--------+------+
|4/1/2014 0:11:00|40.769 |-73.9549|B02512|
|4/1/2014 0:17:00|40.7267|-74.0345|B02512|
|4/1/2014 0:21:00|40.7316|-73.9873|B02512|
|4/1/2014 0:28:00|40.7588|-73.9776|B02512|
|4/1/2014 0:33:00|40.7594|-73.9722|B02512|
+----------------+-------+--------+------+
only showing top 5 rows

+----------------+-------+--------+------+------------------+
|              dt|    lat|     lon|  base|          features|
+----------------+-------+--------+------+------------------+
|4/1/2014 0:11:00| 40.769|-73.9549|B02512| [40.769,-73.9549]|
|4/1/2014 0:17:00|40.7267|-74.0345|B02512|[40.7267,-74.0345]|
|4/1/2014 0:21:00|40.7316|-73.9873|B02512|[40.7316,-73.9873]|
|4/1/2014 0:28:00|40.7588|-73.9776|B02512|[40.7588,-73.9776]|
|4/1/2014 0:33:00|40.7594|-73.9722|B02512|[40.7594,-73.9722]|
+----------------+-------+--------+------+------------------+
only showing top 5 rows

+----

In [11]:
rdd = df.select('lat', 'lon').rdd.map(lambda x: (x[0], x[1]))
clusters = KMeans.train(rdd, 2, maxIterations=10, initializationMode="random")

In [12]:
clusters.centers

[array([ 40.74090043, -73.98586888]), array([ 40.72439491, -73.81897293])]